In [1]:
import numpy as np

import os

import matplotlib.pyplot as plt

import tensorflow as tf

from fcn import FCN
from training_utils import*


In [2]:
path = '../training_data/data/'

images = []

labels = []

for data_path in os.listdir(path):
    
    if os.path.splitext(data_path)[1] == '.npy':
                
        data = np.load(os.path.join(path,data_path), allow_pickle = True)
        
        img = data[:,:,:,0]
        img = img.reshape(img.shape + (1,))
        
        images.append(img)
        
        lbl = data[:,:,:,1:]
        labels.append(lbl)
        
images = np.concatenate(images)
labels = np.concatenate(labels)

In [3]:
images.shape

(90, 256, 256, 1)

In [4]:
labels.shape

(90, 256, 256, 5)

In [5]:
input_shape = (256,256)

num_chemical_elements = 5

output_channels = num_chemical_elements

model = make_model(FCN, input_shape, output_channels)

In [6]:
class R2_CHs(object):

    """
    * R2_CHs: class to calculate the regression metric R^2 between the predicted CHs
    and the true CHs for each chemical element. Input parameters:

    - batch_predictions: batch of predictions maps from the FCN (batch_size, 256,256,5).
    - batch_labels: batch of ground truth maps (batch_size, 256,256,5).
    - num_chemical_elements: number of chemical_elements and output channels of the predictions and labels.

    """

    def __init__(self,batch_predictions,batch_labels,num_chemical_elements = 5):

        self.batch_predictions = batch_predictions

        self.batch_labels = batch_labels

        self.num_chemical_elements = num_chemical_elements

    def get_peaks_pos(self,labels):

        peaks_pos = peak_local_max(labels,min_distance = 1,threshold_abs = 1e-6)

        return peaks_pos


    def get_CHs(self,output, peaks_pos):

        """
        * get_CHs: function to extract the CHs, which are the values of the outputs
        in correspondence of the peaks. Input parameters:

        - output: predictions or labels.
        - peaks_pos: pixel positions of the column's peaks.

        """

        CHs = np.round(output[peaks_pos[:, 0],
                                  peaks_pos[:, 1]])

        return CHs

    def get_r2(self,predictions,labels,peaks_predictions):

        """
        * get_r2: function to calculate the R^2 between the predicted and true CHs for a single element
        Input parameters:

        - predictions: predicted CHs mao of a single element.
        - labels: true CHs map of a single element.
        - peaks_predictions: bool (True or False). If True, the R^2 is calculated in the peaks
          of the predictions, if False the R^2 is calculated in the peaks if the ground truth.

          True: taking into account the false positive.
          False: taking into account the true negative.

          The R^2 is calculated for both the cases and the final value is the average of the two
          If there are less than 2 columns in the prediction, or if the R^2 is negative, the value
          is set to 0.

        """

        if peaks_predictions:

            peaks_pos = self.get_peaks_pos(predictions)

        else:

            peaks_pos = self.get_peaks_pos(labels)

        if len(peaks_pos) > 2:

            CHs_predictions = self.get_CHs(predictions, peaks_pos)

            CHs_labels = self.get_CHs(labels, peaks_pos)

            r2 = r2_score(CHs_predictions,CHs_labels)

            if r2 < 0.0:

                r2 = 0.0
        else:

            r2 = 0.0

        return r2

    def get_avg_r2(self,predictions,labels):

        """
       * get_avg_r2: function to calculate the average of the R^2 between the case
        of CHs calculated in the predicted peaks and the true peaks.

        """

        r2_1 = self.get_r2(predictions,labels,peaks_predictions = True)
        r2_2 = self.get_r2(predictions,labels,peaks_predictions = False)

        avg_r2 = (r2_1 + r2_2)/2

        return avg_r2

    def get_r2_all_elements(self):

        """
        * get_r2_all_elements: function to calculate the R^2 for each element (each output channel)

        """

        r2_all_elements = []

        for i in range(self.num_chemical_elements):

            predictions_single_element = self.predictions[:,:,i]

            labels_single_element = self.labels[:,:,i]

            r2_single_element = self.get_avg_r2(predictions_single_element,
                                                labels_single_element)

            r2_all_elements.append(r2_single_element)

        return r2_all_elements

    def get_r2_all_elements_batch(self):

        """
        * get_r2_all_elements_batch: function to calculate the R^2 for each element
        and for each data in the batch. Once the R^2 is calculated for each element,
        an average R^2 among all the elements is considered.

        """

        r2_1 = 0.0
        r2_2 = 0.0
        r2_3 = 0.0
        r2_4 = 0.0
        r2_5 = 0.0

        num_images = self.batch_predictions
        
        num_images = 8
        
        print(self.batch_predictions.shape)
        
     

       # for self.predictions,self.labels in zip(self.batch_predictions,self.batch_labels):
        for i in range(num_images):
            
            self.predictions = self.batch_predictions[i]
        
            
            self.labels = self.batch_labels[i]
            
           

            r2_all_elements = self.get_r2_all_elements()

            r2_1 += r2_all_elements[0]
            r2_2 += r2_all_elements[1]
            r2_3 += r2_all_elements[2]
            r2_4 += r2_all_elements[3]
            r2_5 += r2_all_elements[4]

        r2_1 = r2_1/num_images
        r2_2 = r2_2/num_images
        r2_3 = r2_3/num_images
        r2_4 = r2_4/num_images
        r2_5 = r2_5/num_images

        average_r2 = (r2_1 + r2_2 + r2_3 + r2_4 + r2_5)/self.num_chemical_elements

        return r2_1,r2_2,r2_3,r2_4,r2_5,average_r2

In [11]:
class R2_CHs_metric(tf.keras.metrics.Metric):
    
    def __init__(self, name="r2", **kwargs):
        super(R2_CHs_metric, self).__init__(name=name, **kwargs)
        self.r2 = self.add_weight(name="r2", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
                
        r2_CHs = R2_CHs(y_pred, y_true)
        r2_all_elements_train = r2_CHs.get_r2_all_elements_batch()
        
        values = r2_all_elements_train[num_chemical_elements]
        
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, self.dtype)
            sample_weight = tf.broadcast_to(sample_weight, values.shape)
            values = tf.multiply(values, sample_weight)
        self.r2.assign_add(tf.reduce_sum(values))

    def result(self):
        
        return self.r2

In [12]:
loss = tf.keras.losses.MeanSquaredError()

model.compile(optimizer = 'adam', loss = loss)

In [10]:
model.fit(images,labels,epochs = 10, batch_size = 8)

Epoch 1/10


TypeError: in user code:

    /opt/anaconda3/envs/tensorflow2.2_python3.7/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /opt/anaconda3/envs/tensorflow2.2_python3.7/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/envs/tensorflow2.2_python3.7/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/envs/tensorflow2.2_python3.7/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/envs/tensorflow2.2_python3.7/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:533 train_step  **
        y, y_pred, sample_weight,